In [ ]:
#installing
!pip install datasets
!pip install bpemb
!pip install stop_words

In [ ]:
#imports

import pandas as pd
import numpy as np
import datasets
from datasets import load_dataset

import nltk
import random
from tqdm.notebook import tqdm
from tqdm import tqdm
tqdm.pandas()
import stop_words

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import torch
from torch.utils.data import Dataset, DataLoader

from bpemb import BPEmb

import re
from transformers import MarianMTModel, MarianTokenizer
import tensorflow as tf

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import LayerNorm
from torch.optim import Adam
from typing import List, Tuple


In [ ]:
#Putting the model to the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## Loading and Splitting the Data

In [ ]:
# Importing the data set
ds = load_dataset("coastalcph/tydi_xor_rc")

# Loading data into train and test
ds_train = load_dataset("coastalcph/tydi_xor_rc", split='train')
ds_val = load_dataset("coastalcph/tydi_xor_rc", split='validation')

In [ ]:
# Creating a validation set from splitting the training data

train_test_split = ds_train.train_test_split(test_size=0.15)

ds_train = train_test_split['train']
validation_dataset = train_test_split['test']

In [ ]:
#Getting the columns we need

#training
train_ru = ds_train.filter(lambda x: x['lang'] == 'ru').select_columns(['context', 'answerable', 'question', 'lang']).to_pandas()
train_ru.reset_index(drop=True, inplace=True)
train_fi = ds_train.filter(lambda x: x['lang'] == 'fi').select_columns(['context', 'answerable', 'question', 'lang']).to_pandas()
train_fi.reset_index(drop=True, inplace=True)
train_ja = ds_train.filter(lambda x: x['lang'] == 'ja').select_columns(['context', 'answerable', 'question' , 'lang']).to_pandas()
train_ja.reset_index(drop=True, inplace=True)

#test set
test_ru = ds_val.filter(lambda x: x['lang'] == 'ru').select_columns(['context', 'answerable', 'question', 'lang']).to_pandas()
test_ru.reset_index(drop=True, inplace=True)
test_fi = ds_val.filter(lambda x: x['lang'] == 'fi').select_columns(['context', 'answerable', 'question', 'lang']).to_pandas()
test_fi.reset_index(drop=True, inplace=True)
test_ja = ds_val.filter(lambda x: x['lang'] == 'ja').select_columns(['context', 'answerable', 'question', 'lang']).to_pandas()
test_ja.reset_index(drop=True, inplace=True)


#validation set
val_ru = validation_dataset.filter(lambda x: x['lang'] == 'ru').select_columns(['context', 'answerable', 'question', 'lang']).to_pandas()
val_ru.reset_index(drop=True, inplace=True)
val_fi = validation_dataset.filter(lambda x: x['lang'] == 'fi').select_columns(['context', 'answerable', 'question', 'lang']).to_pandas()
val_fi.reset_index(drop=True, inplace=True)
val_ja = validation_dataset.filter(lambda x: x['lang'] == 'ja').select_columns(['context', 'answerable', 'question', 'lang']).to_pandas()
val_ja.reset_index(drop=True, inplace=True)

#All language set - used for English
train_data = ds_train.filter(lambda x: x['lang'] == 'ru' or x['lang'] == 'fi' or x['lang'] == 'ja').select_columns(['context', 'answerable', 'question', 'lang']).to_pandas()
test_data = ds_val.filter(lambda x: x['lang'] == 'ru' or x['lang'] == 'fi' or x['lang'] == 'ja').select_columns(['context', 'answerable', 'question', 'lang']).to_pandas()
train_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)
val_data = validation_dataset.filter(lambda x: x['lang'] == 'ru' or x['lang'] == 'fi' or x['lang'] == 'ja').select_columns(['context', 'answerable', 'question', 'lang']).to_pandas()
val_data.reset_index(drop=True, inplace=True)


Filter:   0%|          | 0/13027 [00:00<?, ? examples/s]

Filter:   0%|          | 0/13027 [00:00<?, ? examples/s]

Filter:   0%|          | 0/13027 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2299 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2299 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2299 [00:00<?, ? examples/s]

Filter:   0%|          | 0/13027 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2299 [00:00<?, ? examples/s]

## Methods

In [ ]:
#Adapted from lab exercise 2

#Tokenizes the texts
# texts : text data input (strings)
# tokenizer : the tokenizer of choice
def text_to_batch_bilstm(texts, tokenizer):
    input_ids = []
    seq_lens = []

    for text in texts:
        # Tokenization and encoding
        tokenized_text = tokenizer.encode_ids(text)
        input_ids.append(tokenized_text)

        #storing the length of the tokenized texts
        seq_lens.append(len(tokenized_text))


    #returns input ids and sequence lengths
    return input_ids, seq_lens


In [ ]:
# Adapted from lab exercise 2
# Gets the input ids, sequence lengths, and labels
# pads the input ids and labels
#converts them to tensors and returns the tensors
# input_data : a tuple

def collate_batch_bilstm(input_data: Tuple) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    #Gathering the input ids, sequence lengths, and labels
    input_ids = [i[0][0] for i in input_data]
    seq_lens = [i[1][0] for i in input_data]
    labels = [i[2] for i in input_data]


    # Determining the maximum length in the batch
    max_length = max([len(i) for i in input_ids])

    # Pading input_ids and labels to max length
    #PAD token = 10000
    input_ids = [ids + [10000] * (max_length - len(ids)) for ids in input_ids]
    # Ensuring labels are also lists and also pading them
    labels = [label + [10000] * (max_length - len(label)) if isinstance(label, list) else [10000] * max_length for label in labels]


    # Converting to tensors
    input_ids_tensor = torch.tensor(input_ids, dtype=torch.long)
    seq_lens_tensor = torch.tensor(seq_lens, dtype=torch.long)
    labels_tensor = torch.tensor(labels, dtype=torch.long)

    #returning input ids, sequence lengths, and labels into tensors
    return input_ids_tensor, seq_lens_tensor, labels_tensor


## Embeddings Dictionary

In [ ]:
# Creating a dictionary to store embeddings and vocabularies

#initializing the dictionary
bpemb_dict = {
    'ru': BPEmb(lang="ru", dim=300),
    'ja': BPEmb(lang="ja", dim=300),
    'fi': BPEmb(lang="fi", dim=300),
    'en': BPEmb(lang="en", dim=300),
}

embedding_data = {}

for lang, bpemb in bpemb_dict.items():

    embeddings = bpemb.vectors

    # Adding padding row
    pad_embedding = np.zeros((1, embeddings.shape[1]))
    embeddings = np.concatenate([embeddings, pad_embedding], axis=0)

    # Adding PAD token to vocab
    vocabulary = bpemb.words + ['[PAD]']

    # Converting to tensors
    embeddings_tensor = torch.tensor(embeddings, dtype=torch.float)

    # Creating embedding layer
    embedding_layer = nn.Embedding.from_pretrained(embeddings_tensor, padding_idx=len(vocabulary) - 1)

    # Storing in a dictionary
    embedding_data[lang] = {
        'pretrained_embeddings': embedding_layer,
        'vocabulary': vocabulary
    }

# checking the dictionary
ru_embedding_layer = embedding_data['ru']['pretrained_embeddings']
ru_vocabulary = embedding_data['ru']['vocabulary']
#ru_vocabulary

## LSTM Model

In [ ]:
# Initialzing the dimmension
lstm_dim = 200

class LSTMNetwork(nn.Module):
    def __init__(
            self,
            embedding_dict: dict,
            lstm_dim: lstm_dim,
            dropout_prob: float = 0.1,
    ):
        super(LSTMNetwork, self).__init__()

        self.embedding_dict = embedding_dict

        # Sampling embeddings for dimension extraction
        sample_embedding = list(embedding_dict.values())[0]
        embedding_dim = sample_embedding.embedding_dim
        vocab_size = sample_embedding.num_embeddings

        # Defining the LSTM
        self.model = nn.ModuleDict({
            'lstm': nn.LSTM(
                embedding_dim,
                lstm_dim,
                1,
                batch_first=True,
                dropout=dropout_prob,
                bidirectional=False
            ),

            'output_fc': nn.Linear(lstm_dim, vocab_size)
        })

        # Adding Layer Normalization and Dropout
        self.layer_norm = LayerNorm(lstm_dim)
        self.dropout = nn.Dropout(p=dropout_prob)

        # Initializing weights
        self._init_weights()

    def _init_weights(self):
        for n, p in self.model['lstm'].named_parameters():
            if 'weight' in n:
                nn.init.xavier_normal_(p)
            elif 'bias' in n:
                nn.init.zeros_(p)
        for n, p in self.model['output_fc'].named_parameters():
            if 'weight' in n:
                nn.init.xavier_normal_(p)
            elif 'bias' in n:
                nn.init.zeros_(p)

    def forward(self, inputs, input_lens, lang='en', labels=None, return_perplexity=False):
        # Getting correct lang embeddings
        embeddings = self.embedding_dict[lang].to(inputs.device)
        embeds = embeddings(inputs)

        # Packing padded sequence
        lstm_in = nn.utils.rnn.pack_padded_sequence(
            embeds,
            input_lens.cpu(),
            batch_first=True,
            enforce_sorted=False
        )

        # LSTM output
        lstm_out, hidden = self.model['lstm'](lstm_in)
        lstm_out, _ = nn.utils.rnn.pad_packed_sequence(lstm_out, batch_first=True)

        # Layer Normalization and Dropout
        lstm_out = self.layer_norm(lstm_out)
        lstm_out = self.dropout(lstm_out)

        # Layer for logits
        logits = self.model['output_fc'](lstm_out)

        # Next word prediction
        # Shifting labels and logits
        if labels is not None:
            logits = logits[:, :-1, :].contiguous()
            labels = labels[:, 1:].contiguous()
            logits = logits.view(-1, logits.size(-1))
            labels = labels.view(-1)

            # Calculating cross-entropy loss
            loss_fn = nn.CrossEntropyLoss(ignore_index=embeddings.padding_idx)
            loss = loss_fn(logits, labels)

            # Calculating perplexity if requested
            if return_perplexity:
                perplexity = torch.exp(loss)
                return loss, logits, perplexity
            else:
                return loss, logits
        else:
            return logits


## Training Method

In [ ]:
# Creating the training loop
def train(
    model: nn.Module, #the created model
    train_dl: DataLoader, #Training dataset
    valid_dl: DataLoader, # Validatoin dataset
    optimizer: torch.optim.Optimizer, #Optimizer
    n_epochs: int, # Number of epochs for training
    device: torch.device, # Device to train on
):

  # Keeping track of the loss
  losses = []
  pcounter = 0

  model.to(device)

  # Iterating through epochs
  for ep in range(n_epochs):
    print(ep)
    loss_epoch = []

    # Iterating through each batch
    for batch in tqdm(train_dl):

      # Making the model be in train mode
      model.train()

      # Zeroing out gradiants
      optimizer.zero_grad()

      # Placing the tenors on the device
      batch = tuple(t.to(device) for t in batch)
      input_ids = batch[0]
      seq_lens = batch[1]
      labels = batch[2]


      # Passing the inputs through the model
      # Getting the loss and logits
      loss, logits = model(input_ids, seq_lens, labels=labels)
      losses.append(loss.item())
      loss_epoch.append(loss.item())

      loss.backward()

      # Clipping the gradients
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

      # Updating the weights of the model
      optimizer.step()
      optimizer.zero_grad()

      # Freeing memory or else it crashes
      del input_ids, seq_lens, labels, logits, loss
      torch.cuda.empty_cache()

    # Passing to the evaluation method
    perplexity,_,_ = evaluate(model, valid_dl)

    #printing loss and perplexity
    print(f'train loss: {sum(loss_epoch) / len(loss_epoch)}, perplexity: {perplexity}')

  return model, losses

## Evalutation Method

In [ ]:
# Method to evaluate the model
# Model: given model
# Valid_dl: the data loader for the evaluation dataset
def evaluate(model: nn.Module, valid_dl: DataLoader):

  #Putting model in evaluation mode
  model.eval()
  labels_all = []
  logits_all = []
  total_loss = 0
  total_samples = 0

  # Putting the model to the given device
  model.to(device)

  # Making sure no gradients are accumulated
  with torch.no_grad():
    for batch in tqdm(valid_dl, desc='Evaluation'):
      batch = tuple(t.to(device) for t in batch)
      input_ids = batch[0]
      seq_lens = batch[1]
      labels = batch[2]

      # Putting them all to the device
      seq_lens = seq_lens.to(device).to(torch.int64)
      input_ids = input_ids.to(device)
      labels = labels.to(device)

      # Forward passing through model
      loss, logits = model(input_ids, seq_lens, labels=labels)

      # Accumulating total loss
      batch_size = input_ids.size(0)
      total_loss += loss.item() * batch_size
      total_samples += batch_size

      # Collecting all labels and logits
      labels_all.extend(list(labels.detach().cpu().numpy()))
      logits_all.extend(list(logits.detach().cpu().numpy()))


    # Calculating average loss
    avg_loss = total_loss / total_samples

    # Calculating perplexity
    perplexity = torch.exp(torch.tensor(avg_loss)).item()

    # Deleting to save RAM
    # Will crash if removed
    del input_ids, seq_lens, labels, logits
    torch.cuda.empty_cache()

    return  perplexity, labels_all, logits_all

## English

In [ ]:
#Creating a Dataset Reader for context column

# Initialize BPEmb models
bpemb_ru = BPEmb(lang="ru", dim=300)
bpemb_ja = BPEmb(lang="ja", dim=300)
bpemb_fi = BPEmb(lang="fi", dim=300)
bpemb_en = BPEmb(lang="en", dim=300)

tokenizer_dict = {
    'ru': bpemb_ru,
    'ja': bpemb_ja,
    'fi': bpemb_fi,
    'en': bpemb_en
}

class NextWordPredictionDatasetReader(Dataset):
    def __init__(self, df, tokenizer_dict, max_len):
        """
        Dataset class for next word prediction.

        :param df: A pandas DataFrame containing the dataset.
        :param tokenizer: A tokenizer object that has the method `encode_ids_with_eos`.
        """
        self.df = df
        self.tokenizer_dict = tokenizer_dict
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        question = self.df.loc[idx, 'question']
        context = self.df.loc[idx, 'context']
        lang = self.df.loc[idx, 'lang']

        # Selecting the right tokenizer for language
        tokenizer = self.tokenizer_dict.get(lang, self.tokenizer_dict['en'])

        # Getting the right input text column name
        input_text = context

        # Tokenizes column and returns input ids and sequence lengths
        input_ids, seq_lens = text_to_batch_bilstm([input_text], tokenizer)

        # Splitting input text into words
        words = input_text.split()

        # Determining next word
        if len(words) > 1:
          next_word = words[-1]
        else:
          # If there is one word or no words
          next_word = None

          # Converting the next word to its token
        if next_word:
          # Getting a list of tokens
          label = tokenizer.encode_ids_with_eos(next_word)
        else:
          # Empty list if no next word
          label = []

        return input_ids, seq_lens, label

In [ ]:
lstm_dim = 200

embedding_dict = {
    'ru': embedding_data['ru']['pretrained_embeddings'],
    'ja': embedding_data['ja']['pretrained_embeddings'],
    'fi': embedding_data['fi']['pretrained_embeddings'],
    'en': embedding_data['en']['pretrained_embeddings']
}

# Create the model
model = LSTMNetwork(
    embedding_dict=embedding_dict,
    lstm_dim=lstm_dim
  ).to(device)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


In [ ]:
# Defining hyperparameters
batch_size = 32
lr = 0.001
n_epochs = 8
max_len = 256

tokenizer_dict = {
    'ru': bpemb_ru,
    'ja': bpemb_ja,
    'fi': bpemb_fi,
    'en': bpemb_en
}

# Creating the dataset reader/data loader for training data
train_dataset = NextWordPredictionDatasetReader(train_data, tokenizer_dict, max_len=max_len)
train_dl = DataLoader(train_dataset, batch_size=batch_size,collate_fn=collate_batch_bilstm, shuffle=True, pin_memory=True, num_workers=4)#collate_fn=collate_batch_bilstm

# Creating dataset reader/data loader for validation data
valid_dataset = NextWordPredictionDatasetReader(val_data, tokenizer_dict,max_len=max_len )
valid_dl = DataLoader(valid_dataset, batch_size=10, collate_fn=collate_batch_bilstm, pin_memory=True, num_workers=4)#, collate_fn=collate_batch_bilstm

# Creating the optimizer
optimizer = Adam(model.parameters(), lr=0.001)

# Training
model, losses = train(model, train_dl, valid_dl, optimizer, n_epochs, device)

0


Evaluation: 100%|██████████| 93/93 [00:16<00:00,  5.79it/s]


train loss: 5.123205995836924, perplexity: 72.98271942138672
1


Evaluation: 100%|██████████| 93/93 [00:16<00:00,  5.52it/s]


train loss: 3.961536587670792, perplexity: 61.89029312133789
2


...

train loss: 2.922942118589268, perplexity: 76.80091857910156
6


Evaluation: 100%|██████████| 93/93 [00:16<00:00,  5.54it/s]


train loss: 2.7498084999794186, perplexity: 84.94862365722656
7


Evaluation: 100%|██████████| 93/93 [00:16<00:00,  5.50it/s]


train loss: 2.5894528953141944, perplexity: 95.17669677734375


## Russian

In [ ]:
# New Dataset Reader for question column
# Used for Ru, Fi, and Ja

tokenizer_dict = {
    'ru': bpemb_ru,
    'ja': bpemb_ja,
    'fi': bpemb_fi,
    'en': bpemb_en
}

# Class for next word prediction
class NextWordPredictionDatasetReader(Dataset):
    def __init__(self, df, tokenizer_dict, max_len):

        self.df = df # dataset
        self.tokenizer_dict = tokenizer_dict # The dictionary with all the tokenizers
        self.max_len = max_len # Max length used for padding sequences

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        question = self.df.loc[idx, 'question']
        context = self.df.loc[idx, 'context']
        lang = self.df.loc[idx, 'lang']

        # Selecting tokenizer based on language
        # Defaults to English
        tokenizer = self.tokenizer_dict.get(lang, self.tokenizer_dict['en'])

        # Geting input IDs and sequence lengths using the tokenizer
        input_ids, seq_lens = text_to_batch_bilstm([question], tokenizer)

        # Next word prediction
        # Splitting data
        words = question.split()
        next_word = words[-1] if len(words) > 1 else words[0] if words else None

        # Encoding the next word to token IDs
        if next_word:
            label = tokenizer.encode_ids(next_word)
        else:
          # Using the padding ID for cases with no next word
            label = [10000] * self.max_len

        return input_ids, seq_lens, label


In [ ]:
lstm_dim = 200

embedding_dict = {
    'ru': embedding_data['ru']['pretrained_embeddings'],
    'ja': embedding_data['ja']['pretrained_embeddings'],
    'fi': embedding_data['fi']['pretrained_embeddings'],
    'en': embedding_data['en']['pretrained_embeddings']
}

# Creating the model
model = LSTMNetwork(
    embedding_dict=embedding_dict,
    lstm_dim=lstm_dim
  ).to(device)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


In [ ]:
# Defining hyperparameters
batch_size = 32
lr = 0.001
n_epochs = 8
max_len = 256

tokenizer_dict = {
    'ru': bpemb_ru,
    'ja': bpemb_ja,
    'fi': bpemb_fi,
    'en': bpemb_en
}

# Creating the dataset readers for Russian training set
train_dataset = NextWordPredictionDatasetReader(train_ru, tokenizer_dict, max_len=max_len)
train_dl = DataLoader(train_dataset, batch_size=batch_size,collate_fn=collate_batch_bilstm, shuffle=True, pin_memory=True, num_workers=4)

#Creating dataset reader for russian validation dataset
valid_dataset = NextWordPredictionDatasetReader(val_ru, tokenizer_dict,max_len=max_len )
valid_dl = DataLoader(valid_dataset, batch_size=batch_size, collate_fn=collate_batch_bilstm, pin_memory=True, num_workers=4)

# Creating the optimizer
optimizer = Adam(model.parameters(), lr=0.001)

# Training
model, losses = train(model, train_dl, valid_dl, optimizer, n_epochs, device)

0


Evaluation: 100%|██████████| 9/9 [00:00<00:00, 20.75it/s]


train loss: 8.672563773614389, perplexity: 2318.3046875
1


Evaluation: 100%|██████████| 9/9 [00:00<00:00, 20.33it/s]


train loss: 5.931706808231495, perplexity: 2070.6181640625
2


...

train loss: 2.3574093690624944, perplexity: 4206.98095703125
6


Evaluation: 100%|██████████| 9/9 [00:00<00:00, 21.50it/s]


train loss: 1.9635811227339286, perplexity: 5668.56689453125
7


Evaluation: 100%|██████████| 9/9 [00:00<00:00, 20.45it/s]

train loss: 1.7474813306773151, perplexity: 7008.5361328125


## Finnish


In [ ]:
lstm_dim = 200

embedding_dict = {
    'ru': embedding_data['ru']['pretrained_embeddings'],
    'ja': embedding_data['ja']['pretrained_embeddings'],
    'fi': embedding_data['fi']['pretrained_embeddings'],
    'en': embedding_data['en']['pretrained_embeddings']
}

# Creating the model
model = LSTMNetwork(
    embedding_dict=embedding_dict,
    lstm_dim=lstm_dim
  ).to(device)

In [ ]:
# Defining hyperparameters
batch_size = 32
lr = 0.001
n_epochs = 8
max_len = 256

# Creating the dataset readers for Finnish training dataset
train_dataset = NextWordPredictionDatasetReader(train_fi, tokenizer_dict, max_len=max_len)
train_dl = DataLoader(train_dataset, batch_size=batch_size,collate_fn=collate_batch_bilstm, shuffle=True, pin_memory=True, num_workers=4)

# Creating dataset reader for Finnish validation dataset
valid_dataset = NextWordPredictionDatasetReader(val_fi, tokenizer_dict,max_len=max_len )
valid_dl = DataLoader(valid_dataset, batch_size=batch_size, collate_fn=collate_batch_bilstm, pin_memory=True, num_workers=4)

# Creating the optimizer
optimizer = Adam(model.parameters(), lr=0.001)

# Training
model, losses = train(model, train_dl, valid_dl, optimizer, n_epochs, device)

0


Evaluation: 100%|██████████| 11/11 [00:00<00:00, 22.68it/s]


train loss: 5.480512347137719, perplexity: 83.65003967285156
1


Evaluation: 100%|██████████| 11/11 [00:00<00:00, 21.91it/s]


train loss: 3.542366567410921, perplexity: 77.82254028320312
2


...

train loss: 2.3600230447032997, perplexity: 114.94468688964844
6


Evaluation: 100%|██████████| 11/11 [00:00<00:00, 21.96it/s]


train loss: 2.221009039042289, perplexity: 124.29462432861328
7


Evaluation: 100%|██████████| 11/11 [00:00<00:00, 21.27it/s]

train loss: 2.1191319101735164, perplexity: 133.4221954345703


## Japanese

In [ ]:
#increasing the dimension to work better for Jappanese
lstm_dim = 200

# initializing the embedding dictionary
embedding_dict = {
    'ru': embedding_data['ru']['pretrained_embeddings'],
    'ja': embedding_data['ja']['pretrained_embeddings'],
    'fi': embedding_data['fi']['pretrained_embeddings'],
    'en': embedding_data['en']['pretrained_embeddings']
}

# Creating the model
model = LSTMNetwork(
    embedding_dict=embedding_dict,
    lstm_dim=lstm_dim
  ).to(device)

In [ ]:
# Defining the hyperparameters
batch_size = 32
lr = 0.001
n_epochs = 8
max_len = 128


# Creating the dataset readers for Japanese training data
train_dataset = NextWordPredictionDatasetReader(train_ja, tokenizer_dict, max_len=max_len)
train_dl = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_batch_bilstm, shuffle=True, pin_memory=True, num_workers=4)

# Creating dataset reader for Japanese validation data
valid_dataset = NextWordPredictionDatasetReader(val_ja, tokenizer_dict,max_len=max_len )
valid_dl = DataLoader(valid_dataset, batch_size=batch_size, collate_fn=collate_batch_bilstm, pin_memory=True, num_workers=4)

# Creating the optimizer
optimizer = Adam(model.parameters(), lr=0.001)

# Training
model, losses = train(model, train_dl, valid_dl, optimizer, n_epochs, device)

0


Evaluation: 100%|██████████| 11/11 [00:00<00:00, 19.58it/s]


train loss: 7.2031062187687045, perplexity: 507.9928283691406
1


Evaluation: 100%|██████████| 11/11 [00:00<00:00, 21.04it/s]


train loss: 5.549883557904151, perplexity: 325.0409851074219
2


...

train loss: 3.2574638820463613, perplexity: 226.73358154296875
6


Evaluation: 100%|██████████| 11/11 [00:00<00:00, 19.51it/s]


train loss: 2.8571630600960023, perplexity: 230.9124298095703
7


Evaluation: 100%|██████████| 11/11 [00:00<00:00, 19.30it/s]

train loss: 2.509425013296066, perplexity: 242.28753662109375
